# Computational methods - integrating
Written by Michael Perviy AKA @itdaos

Main tasks:
* Numericaly solve defined integrals
    * First integral
$$
\int\limits_{0}^{0.5} e^{-\frac{3}{25}x^2} dx
$$
    * Second integral
$$
\iint\limits^{D} (12x^2y^2 + 16x^3y^3) \, dxdy \qquad D: \; x = 1; \; y=x^2; \; y = -\sqrt{x}
$$

In [91]:
import numpy as np
import math
# Constants
N = 1000
EPS = 0.2
INT_GOAL1 = 0.495045
INT_GOAL2 = 1

# functions to be integrated
def f(x: float) -> float:
    return math.e ** (-3/25 * (x**2) )

def g(x: float, y: float) -> float:
    return 12*(x**2)*(y**2) + 16*(x**3)*(y**3)

def g_star(x: float, y: float) -> float:
    if 0 <= x <= 1 and -math.sqrt(x) <= y <= x**2:
        return g(x, y)
    else:
        return 0.0
# tests
assert round(f(0.5), 3) == 0.970
assert g(0.75, 0.5) == 2.53125
assert g_star(0.75, 0.5) == 2.53125

# integration algos
def int_by_rect(func, a, b, n=N):
    h = (a + b)/n
    vals = np.linspace(a, b, n)
    func = np.vectorize(func)
    vals = func(vals)

    return h*np.sum(vals)

def int_by_trap(func, a, b, n=N):
    h = (a + b)/n
    x1 = np.linspace(a, b-h, n)
    x2 = np.linspace(a+h, b, n)
    func = np.vectorize(func)
    y = (func(x1) + func(x2))
    return h/2 * np.sum(y)

def int_by_parb(func, a, b, n=N):
    h = (a + b)/n
    x1 = np.linspace(a, b-2*h, n)
    x2 = np.linspace(a+h, b-h, n)
    x3 = np.linspace(a+2*h, b, n)
    func = np.vectorize(func)
    y = (func(x1) + func(x2) + func(x3) )
    return h/3 * np.sum(y)

def iint_by_simp(func, x_0, x_1, y_0, y_1, n=N):
    h = (x_1 - x_0)/n
    k = (y_1 - y_0)/n
    
    x = np.linspace(x_0, x_1, n+1)
    y = np.linspace(y_0, y_1, n+1)
    
    op = np.empty((n+1, n+1))
    
    l = np.ones(n+1)
    for i in range(n+1):
        if i != 0 and i != n:
            l[i] = 4 if i % 2 == 1 else 2
    l = np.mat(l)
    l = l.T.dot(l)
    
    for i in range(n+1):
        for j in range(n+1):
            op[i, j] = l[i, j] * func(x[i], y[j])
    
    return k*h/9 * np.sum(op)
    

In [92]:
print("First integral calculations:")
print("Real value = ", INT_GOAL1)
res = int_by_rect(f, 0, 0.5)
relative_error = (abs(INT_GOAL1 - res)/INT_GOAL1) * 100
print("Rect algo # res = %f \n\tRelative error = %f" %  (res, relative_error), "%")
res = int_by_trap(f, 0, 0.5)
relative_error = (abs(INT_GOAL1 - res)/INT_GOAL1) * 100
print("Trap algo # res = %f \n\tRelative error = %f" %  (res, relative_error), "%")
res = int_by_parb(f, 0, 0.5)
relative_error = (abs(INT_GOAL1 - res)/INT_GOAL1) * 100
print("Parb algo # res = %f \n\tRelative error = %f" %  (res, relative_error), "%")

print("\nSecond integral calculations:")
x_0, x_1, y_0, y_1 = 0, 1, -1, 1
res = iint_by_simp(g_star, x_0, x_1, y_0, y_1)
relative_error = (abs(INT_GOAL2 - res)/INT_GOAL2) * 100
print("Simpson algo # res = %f \n\tRelative error = %f" %  (res, relative_error), "%")

First integral calculations:
Real value =  0.495045
Rect algo # res = 0.495042 
	Relative error = 0.000557 %
Trap algo # res = 0.495045 
	Relative error = 0.000065 %
Parb algo # res = 0.495047 
	Relative error = 0.000426 %

Second integral calculations:
Simpson algo # res = 1.000607 
	Relative error = 0.060698 %
